In [1]:
import json
import gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing, tree
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [2]:
df = pd.read_csv('pet_supplies.csv')
df

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,3,2,True,"12 2, 2016",A2KN4FJVI2TZSF,972585419,M.G.,I purchased this cd for my Pocket Parrot. It h...,It's okay,1480636800,NaN,NaN
1,3,NaN,True,"10 17, 2016",A3RMA1DD66JDRV,972585419,Susan P.,"Maybe it's just my Amazon parrot, but she's no...",It might work for a different bird,1476662400,NaN,NaN
2,5,2,True,"09 9, 2016",AAANYRIEOIT3R,972585419,Luggie Lugnut,I bought this to help me teach my blue quaker ...,I bought this to help me teach my blue quaker ...,1473379200,NaN,NaN
3,3,NaN,True,"08 31, 2016",A3W44VX0LXAOHU,972585419,Melissa Partin,Bird showed no interest in it....,Three Stars,1472601600,NaN,NaN
4,2,NaN,True,"08 18, 2016",A16ZDBZGKYDRSU,972585419,Cheryle Bell,Didn't do a thing for my African Grey! He hate...,He hated it and it was very obnoxious,1471478400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
109108,5,NaN,True,"09 12, 2015",A1MJU7YAJMTWPS,B0002ARWTA,shopsandcoupons,"Very handy, using it all the time now. Fits on...",Great buy,1442016000,"{'Size:': ' Covers Are 3 1/2"" Diameter'}",NaN
109109,5,NaN,True,"07 18, 2015",A1NK6UXWXKFMC7,B0002AR15U,Just Lucky,Best dog toys ever. Fill with a little yogurt...,Best dog toys ever,1437177600,"{'Size:': ' X-Large', 'Package Type:': ' Stand...",NaN
109110,4,NaN,True,"07 16, 2015",A2Q5AHXHSZVUJE,B0002AR15U,alberto almada,great,Four Stars,1437004800,"{'Size:': ' Medium', 'Package Type:': ' Standa...",NaN
109111,4,NaN,True,"07 15, 2015",A2S2NKJ530LNTH,B0002AR15U,Jesus,Very durable! awesome how it bounces every whi...,awesome how it bounces every which way never k...,1436918400,"{'Size:': ' Medium', 'Package Type:': ' Standa...",NaN


In [3]:
# convert rating to sentiments
def convert_sentiments(rating):
    if (rating == 5) or (rating == 4):
        return "Positive"
    elif rating == 3:
        return "Neutral"
    elif (rating == 2) or (rating == 1):
        return "Negative"

# Add sentiments to the data
df["Sentiment"] = df["overall"].apply(convert_sentiments)

In [4]:
df.groupby('Sentiment').size()

Sentiment
Negative    10637
Neutral      8381
Positive    90095
dtype: int64

In [5]:
df['reviewText']

0         I purchased this cd for my Pocket Parrot. It h...
1         Maybe it's just my Amazon parrot, but she's no...
2         I bought this to help me teach my blue quaker ...
3                         Bird showed no interest in it....
4         Didn't do a thing for my African Grey! He hate...
                                ...                        
109108    Very handy, using it all the time now. Fits on...
109109    Best dog toys ever.  Fill with a little yogurt...
109110                                                great
109111    Very durable! awesome how it bounces every whi...
109112                           need more. Best stuff ever
Name: reviewText, Length: 109113, dtype: object

In [6]:
df['reviewText'] = df['reviewText'].str.lower()

In [7]:
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42, shuffle = True)


In [8]:
l = ['great','love', 'perfect', 'favorite', 'bad', 'dislike', 'hate']
df_encoded_train = train_df.join(
pd.get_dummies(train_df['reviewText'].str.extract(f"({'|'.join(l)})",expand=False)))

df_encoded_test = test_df.join(
pd.get_dummies(test_df['reviewText'].str.extract(f"({'|'.join(l)})",expand=False)))
df_encoded_train

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,bad,dislike,favorite,great,hate,love,perfect
79604,5,NaN,True,"02 4, 2018",A23II8BC51BBHV,B0002APSKA,Robster,works great on aquariums less than 10 gallons.,Works great on my 2 gallon tank,1517702400,NaN,NaN,Positive,0,0,0,1,0,0,0
17848,5,8,False,"05 30, 2012",A27L3LYLHCQZYG,B000084EYK,Amberabcg,"i have 5-7 dogs at any given time, sometimes f...",Excellent Quality; my dogs LOVE it!,1338336000,NaN,NaN,Positive,0,0,0,0,0,1,0
49296,5,NaN,True,"02 25, 2016",A3JE5D7OODKVCU,B0002565KC,Rowan,this is a great filter for small tanks. i used...,Will likely buy again,1456358400,NaN,NaN,Positive,0,0,0,1,0,0,0
89553,5,NaN,True,"08 16, 2017",A3485ZQ6BLH2L8,B0002AQN2M,Kittymama,use it on my cats behind when it isn't as.frag...,Works for me,1502841600,{'Size:': ' 1Pack'},NaN,Positive,0,0,0,0,0,0,0
69259,4,NaN,True,"11 16, 2016",A19BB346CLDCYE,B00027469M,chris,this does its job well. it is just the right s...,Does exactly as it is suppose to. For the pric...,1479254400,{'Size:': ' Small'},NaN,Positive,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54886,5,NaN,True,"03 15, 2016",A37QAYBHYR8ZQ2,B000255NCI,Amydionne,simply most import thing needed for monitoring...,Mandatory Item,1458000000,{'Style:': ' Reef'},NaN,Positive,0,0,0,0,0,0,0
76820,5,NaN,True,"04 27, 2015",A1XD0NKTQPNTUX,B0002A5VK2,Heather Henry,so awesome. over a year later and it still rec...,So awesome. Over a year later and it still rec...,1430092800,{'Size:': ' 100 MILLILITER'},NaN,Positive,0,0,0,0,0,0,0
103694,4,NaN,True,"06 23, 2016",A3B2Q6YB0YX8O0,B0002ARHI6,Carol P.,perfect fit! still trying to get my puppy used...,Perfect fit! Still trying to get my puppy used...,1466640000,"{'Size:': ' Small', 'Color:': ' Bright Pink'}",NaN,Positive,0,0,0,0,0,0,1
860,4,NaN,True,"04 23, 2016",A12RVDQ78IJO6K,B00004T2WR,Christopher Smith,healthy snacks and my dog loves them,Dog loves em!,1461369600,NaN,NaN,Positive,0,0,0,0,0,1,0


In [9]:
X_train = df_encoded_train.iloc[:, 16:]
Y_train = df_encoded_train['Sentiment']
X_test = df_encoded_test.iloc[:, 16:]
Y_test = df_encoded_test['Sentiment']

In [10]:
clf = DecisionTreeClassifier()

vec = DictVectorizer()
#X_train = vec.fit_transform(X_train).toarray()
#X_test = vec.fit_transform(X_test).toarray()

le = preprocessing.LabelEncoder()
le.fit(['Positive', 'Negative', 'Neutral'])
Y_train = le.transform(train_df[['Sentiment']])
Y_test = le.transform(test_df[['Sentiment']])

clf = clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)

accuracy_score(Y_test, y_pred)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8290794116299317

In [11]:
accuracy_score(Y_test, y_pred)

0.8290794116299317

In [12]:
rf = RandomForestClassifier()

rf = rf.fit(X_train, Y_train)
y_pred = rf.predict(X_test)


In [13]:
accuracy_score(Y_test, y_pred)

0.8290794116299317

In [14]:
df_results = pd.DataFrame(Y_test, columns=['Actual Sentiment'])
df_results['Predicted Sentiment'] = y_pred

In [15]:
df_results.groupby('Actual Sentiment').size()

Actual Sentiment
0     2060
1     1670
2    18093
dtype: int64

In [16]:
df_results.groupby('Predicted Sentiment').size()

Predicted Sentiment
2    21823
dtype: int64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b7614770-cffc-4a92-89aa-4beb219b034f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>